In [1]:
import os
import numpy as np
import pandas as pd
import faiss
from tqdm import tqdm

## Data Loading

Load yelp dataset along with pre-computed user and item vectors.

In [2]:
# read datasets
df_item = pd.read_json('../data/generated/item.json')
df_user = pd.read_json('../data/generated/user.json')

In [3]:
df_review = pd.read_csv('../data/yelp_data/review.csv')

In [4]:
df_item.head(n=1)

,item_id,name,city,state,postal_code,latitude,longitude,stars,aspect_weights
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",Cuyahoga Falls,OH,44221,41.119535,-81.47569,3.5,"[0.0005837712, 0.0022712347000000003, 0.0, 0.0..."


In [5]:
df_user.head(n=1)

,user_id,name,review_count,yelping_since,aspect_weights
0,JJ-aSuM4pCFPdkfoZ34q0Q,Chris,10,2013-09-24,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [6]:
df_review.head(n=1)

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,vkVSCC7xljjrAI4UGfnKEQ,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5,2016-05-28,Super simple place but amazing nonetheless. It...,0,0,0


## Preparations

Load index for fast nearby user retrieval and function for converting user information to feature vectors.

In [7]:
# initialize the user index
d = len(df_user.iloc[0]['aspect_weights'])
index = faiss.IndexFlatIP(d)

# populate the index with user vectors
user_vectors = np.array(df_user['aspect_weights'].to_list()).astype(np.float32)
user_vectors = user_vectors / np.linalg.norm(user_vectors, axis=1, keepdims=True)
index.add(user_vectors)

/data/jingyuny/projects/yelp/venv/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


In [8]:
def get_user_vector(reviews):
    # sample input: [{"item_id": "xyz", "rating": 5}]
    # output: user vector
    w_uk = np.zeros((d,))
    for review in reviews:
        item_row = df_item.loc[df_item["item_id"] == review["item_id"]]
        if len(item_row) == 0:
            continue
        w_uk += float(review["rating"]) * np.array(item_row.iloc[0]["aspect_weights"]).astype(float)
    return w_uk / np.linalg.norm(w_uk)

In [9]:
user_vector = get_user_vector([{"item_id": "XOSRcvtaKc_Q5H1SAzN20A", "rating": 5}])

## Item Rating Prediction

Inspired by section 4.2.2 of [1], we design our method for predicting ratings given (user, item) pairs. The rough procedure for predicting the ratings is described as follows:

* We denote the target user and item as $u_m$ and $i_n$.
* We find a list of all neighboring users $N_k(u_m)$ of the target user, where distance is measured by cosine similarity.
* For each nearby user $u'$, we find all reviews $(u', i', r) \in R(u')$ made by this user.
* For each user, we save the tuple $(\text{sim}(u_m, u'), \text{sim}(i_n, i'), r)$ in a list `score_info`.
* We compute the predicted score using the `score_info` list by taking the weighted some of the scores $r$ by weights $\text{sim}(u_m, u') \cdot \text{sim}(i_n, i')$.

Concerely, we use the following formula for item score prediction: $$ \hat{r}(u_m, i_n) = \frac{\sum_{u' \in N_k(u_m)} \sum_{(u', i', r) \in R(u')} \text{sim}(u_m, u') \cdot \text{sim}(i_n, i') \cdot r}{\sum_{u' \in N_k(u_m)} \sum_{(u', i', r) \in R(u')} \text{sim}(u_m, u') \cdot \text{sim}(i_n, i')}.$$

[1] Hernández-Rubio, M., Cantador, I., Bellogín, A. 2019. A comparative analysis of recommender systems based on item aspect opinions extracted from user reviews. User Modeling and User-Adapted Interaction 29(2), 381-441.

In [10]:
def rate_item(user_vector, item_id, k=10):
    # fetch item vector
    item_vector = np.array(df_item[df_item["item_id"] == item_id].iloc[0]["aspect_weights"]).astype(float)

    # get the nearest k users given the user vector
    nearest_user_dists, nearest_users = index.search(user_vector.reshape((1,d)).astype(np.float32), k)
    nearest_user_dists = nearest_user_dists / np.linalg.norm(nearest_user_dists)

    score_info = []

    # for each nearby user...
    for user_ix in nearest_users[0]:
        # get his/her similarity value the queried user
        user_id = df_user.iloc[user_ix]["user_id"]
        nearby_vector = np.array(df_user[df_user["user_id"] == user_id].iloc[0]["aspect_weights"]).astype(float)
        user_sim = np.dot(user_vector, nearby_vector)/(np.linalg.norm(user_vector) * np.linalg.norm(nearby_vector))

        # iterate all his/her reviews
        reviews = df_review[df_review["user_id"] == user_id]
        for _, review in reviews.iterrows():
            # record the (user similarity, item similarity, reviewed score) tuple
            item = df_item[df_item["item_id"] == review["business_id"]]
            if len(item) == 0:
                continue
            review_vector = np.array(item.iloc[0]["aspect_weights"]).astype(float)
            item_sim = np.dot(item_vector, review_vector)/(np.linalg.norm(item_vector) * np.linalg.norm(review_vector))
            score_info.append([user_sim, item_sim, review["stars"]])

    # accumulate results
    weights = np.array([x[0] * x[1] for x in score_info])
    weights /= np.sum(weights)
    ratings = np.array([x[2] for x in score_info])
    return weights.dot(ratings)

In [11]:
rate_item(user_vector, "XOSRcvtaKc_Q5H1SAzN20A")

2.3000000722859193

In [12]:
def predict_score(user, item_id):
    # sample user input: { 
    #     "latitude": 40,
    #     "longitude": -80,
    #     "...": (other info),
    #     "reviews": [
    #         {
    #             "item_id": "A",
    #             "rating": 4.5
    #         }
    #     ]
    # }
    return rate_item(get_user_vector(user["reviews"]), item_id)

## Process Reviews

Clean up the reviews dataframe and save to `reviews.json`.

In [218]:
df_item_to_join = df_item[['aspect_weights', 'item_id']].rename(columns={'aspect_weights': 'business_vec'}).set_index('item_id')
df_user_to_join = df_user[['aspect_weights', 'user_id']].rename(columns={'aspect_weights': 'user_vec'}).set_index('user_id')

In [219]:
df_review_join = df_review.drop(columns=['useful', 'funny', 'cool', 'date']).join(df_item_to_join, on='business_id').join(df_user_to_join, on='user_id').dropna()

In [220]:
user_avg = df_review_join.groupby(['user_id'])[['user_id', 'stars']].mean().rename(columns={'stars': 'user_avg_stars'})
df_review_join = df_review_join.join(user_avg, on='user_id')

In [221]:
df_review_join = df_review_join.reset_index().drop(columns=['index'])

In [222]:
drop_business_vec = np.array(df_review_join['business_vec'].tolist(), dtype=float)
drop_business_vec = np.sum(np.isnan(business_vec), axis=1).astype(int)
drop_user_vec = np.array(df_review_join['user_vec'].tolist(), dtype=float)
drop_user_vec = np.sum(np.isnan(user_vec), axis=1).astype(int)
drop_indices = np.logical_or(drop_business_vec, drop_user_vec)

In [223]:
df_review_join['drop_flag'] = pd.Series(drop_indices)

In [224]:
df_review_join = df_review_join[df_review_join['drop_flag'] == False].drop(columns=['drop_flag'])
df_review_join = df_review_join.reset_index().drop(columns=['index'])

In [225]:
df_review_join.tail()

,review_id,user_id,business_id,stars,text,business_vec,user_vec,user_avg_stars
3217171,evfB2Xzw4u7aYftWClQfeg,mPjPyipaD0C_myqWqDipZg,--SrzpvFLwP_YFwB_Cetow,2,"So after reading Precilla C. review, I decided...","[0.0004520796, 0.020557723400000002, 0.0, 0.0,...","[0.11093226190000001, 0.7127692474, 0.00199766...",2.818182
3217172,-CJNPrDWgIkorx4iEZJXIg,mPjPyipaD0C_myqWqDipZg,pOEL97ld-FJMKO8Ki8JmYg,3,"Spring rolls was pretty good, cod was a bit ra...","[0.0032446463, 0.0064399176, 0.0, 0.0, 0.00132...","[0.11093226190000001, 0.7127692474, 0.00199766...",2.818182
3217173,W9eVvOcpBvG6lpJPoJOxuA,mPjPyipaD0C_myqWqDipZg,5ubokMNw8qfbX2WtxgJG1Q,4,"Had a 8 dish set meal, was enough for 10 peopl...","[0.0005274262000000001, 0.009520718400000001, ...","[0.11093226190000001, 0.7127692474, 0.00199766...",2.818182
3217174,hqQ1UTFKMN2P1ezUow48OQ,mPjPyipaD0C_myqWqDipZg,EO3i5kTUG7_S2OIQ23sdSA,3,"A small, cozy family run Authentic korean rest...","[0.0008048290000000001, 0.046277666, 0.0, 0.0,...","[0.11093226190000001, 0.7127692474, 0.00199766...",2.818182
3217175,ldsIs3sGXPJ7WM7VyAm4lQ,mPjPyipaD0C_myqWqDipZg,HloEEXm_QzEpycuwA_RtQA,3,Really nice atmosphere and settings and very c...,"[0.000623053, 0.0690150971, 0.0, 0.0, 0.002973...","[0.11093226190000001, 0.7127692474, 0.00199766...",2.818182


In [226]:
df_review_join.to_json('../data/generated/review.json')